In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import model_from_json
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
import random
import os
import cv2

import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/ext/Data/ypw/driving_log.csv', skipinitialspace=True)
df.columns = ['center', 'left', 'right', 'dir', 'acc', 'nacc', 'speed']
df = df.drop((df[df['acc'] != 1]).index)
df = df.drop((df[df['speed'] < 30.1]).index)
print(df['speed'].mean(), df['dir'].mean(), df['dir'].max(), df['dir'].min())
df = df.reset_index(drop=True)
df['dir'] = df['dir'] * 100
df.head()

30.186262807718055 -0.004452897419339064 1.0 -1.0


,center,left,right,dir,acc,nacc,speed
0,center_2017_01_05_10_33_07_558.jpg,left_2017_01_05_10_33_07_558.jpg,right_2017_01_05_10_33_07_558.jpg,-46.68390,1.0,0.0,30.14583
1,center_2017_01_05_10_33_07_666.jpg,left_2017_01_05_10_33_07_666.jpg,right_2017_01_05_10_33_07_666.jpg,-82.77869,1.0,0.0,30.10305
2,center_2017_01_05_10_33_07_769.jpg,left_2017_01_05_10_33_07_769.jpg,right_2017_01_05_10_33_07_769.jpg,-51.98100,1.0,0.0,30.18893
3,center_2017_01_05_10_33_07_871.jpg,left_2017_01_05_10_33_07_871.jpg,right_2017_01_05_10_33_07_871.jpg,-21.23459,1.0,0.0,30.19710
4,center_2017_01_05_10_33_07_987.jpg,left_2017_01_05_10_33_07_987.jpg,right_2017_01_05_10_33_07_987.jpg,0.00000,1.0,0.0,30.18751


In [3]:
df = df.drop(['left', 'right', 'acc', 'nacc', 'speed'], axis=1)
df.columns = ['filename', 'dir']
df.head()

,filename,dir
0,center_2017_01_05_10_33_07_558.jpg,-46.68390
1,center_2017_01_05_10_33_07_666.jpg,-82.77869
2,center_2017_01_05_10_33_07_769.jpg,-51.98100
3,center_2017_01_05_10_33_07_871.jpg,-21.23459
4,center_2017_01_05_10_33_07_987.jpg,0.00000


In [4]:
import cv2
import numpy as np
import random

class Smooth:
    def __init__(self, windowsize=50):
        self.window_size = windowsize
        self.data = np.zeros(self.window_size, dtype=np.float32)
        self.index = 0
    
    def __iadd__(self, x):
        self.data[self.index % self.window_size] = x
        self.index += 1
        return self

    def __str__(self):
        return str(self.data[:self.index].mean())
    
    def __float__(self):
        return float(self.data[:self.index].mean())

class BatchGenerator:
    def __init__(self, df, batch, image_shape):
        self.df = df
        self.batch = batch
        self.image_shape = image_shape
        self.num = len(self.df.index)
    
    def next(self):
        X = np.ndarray((self.batch, self.image_shape[1], self.image_shape[0], 3), dtype=np.float32)
        y = np.ndarray((self.batch, 1), dtype=np.float32)
        image_shape = self.image_shape
        
        for i, index in enumerate(random.sample(self.df.index.tolist(), self.batch)):
            img = cv2.imread("/ext/Data/ypw/IMG/" + self.df.loc[index]['filename'])
            img = cv2.resize(img, image_shape)
            X[i] = img
            y[i] = self.df.loc[index]['dir']

        return X, y

print("Done")

Done


In [5]:
batch = 64
image_shape = (128, 128)

df_train, df_val = train_test_split(df, test_size=0.05, random_state=42)
train = BatchGenerator(df_train, batch, image_shape)
val = BatchGenerator(df_val, batch, image_shape)

print(train.num, val.num)

21417 1128


In [6]:
from keras.applications import VGG16

img_input = Input(shape=(image_shape[1], image_shape[0], 3))
base_model = VGG16(weights='imagenet', input_tensor=img_input, include_top=False)

In [7]:
x = base_model.output

x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1)(x)

model = Model(input=base_model.input, output=x)

model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

/home/jidou/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [9]:
zip(range(len(model.layers)), model.layers)

In [10]:
val_num = len(df_val.index)
X_valid = np.ndarray((val_num, image_shape[1], image_shape[0], 3), dtype=np.float32)
y_valid = np.ndarray((val_num, 1), dtype=np.float32)
for i, index in enumerate(df_val.index):
    img = cv2.imread("/ext/Data/ypw/IMG/" + df_val.loc[index]['filename'])
    img = cv2.resize(img, image_shape)
    X_valid[i] = img
    y_valid[i] = df_val.loc[index]['dir']

def validation():
    loss, acc = model.evaluate(X_valid, y_valid, verbose=0)
    return acc, loss

acc,loss = validation()
print("acc={} loss={}".format(acc,loss))

acc=0.025709219858156027 loss=339.80700250720304


In [11]:
def train_one_batch(generator):
    acc = Smooth()
    loss = Smooth()
    batch_num = generator.num//batch
    pbar = tqdm(range(batch_num))
    for i in pbar:
        X,y = generator.next()
        _loss, _acc = model.train_on_batch(X, y)
        acc += _acc
        loss += _loss
        
        if i != batch_num-1:
            pbar.set_description("train loss: %.4f" % loss)
        else:
            pbar.set_description("train loss: %.4f\t valid loss: %.4f\t" % (loss, validation()[1]))

def transfer_learning(top_num=-4, epoch=10, optimizer='adadelta'):
    for layer in model.layers[:top_num]:
        layer.trainable = False

    for layer in model.layers[top_num:]:
        layer.trainable = True
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
    for i in range(epoch):
        train_one_batch(train)
        
print("Done")

Done


In [17]:
transfer_learning(18, epoch=6)

train loss: 98.8203	 valid loss: 165.1724	: 100%|██████████| 334/334 [00:49<00:00,  1.74it/s]


In [12]:
transfer_learning(14, epoch=10)

train loss: 72.3272	 valid loss: 199.9035	: 100%|██████████| 334/334 [01:09<00:00,  1.34it/s]


In [13]:
X_val, y_val = val.next()
y_pred = model.predict(X_val)
delta = y_val - y_pred

zip(y_val.tolist(), y_pred.tolist())

In [14]:
model.save('vgg16_model.h5', overwrite=True)
print("saved")

saved


In [15]:
from keras.applications import VGG16

img_input = Input(shape=(image_shape[1], image_shape[0], 3))
base_model = VGG16(weights='imagenet', input_tensor=img_input, include_top=False)

x = base_model.output

x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1)(x)

model = Model(input=base_model.input, output=x)

model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

with open('vgg16_model.json', 'w') as f:
    f.write(model.to_json())
    print("json writed")

json writed


/home/jidou/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
